In [5]:
# from Crypto.PublicKey import RSA

# # Generate RSA keys
# private_key = RSA.generate(2048)
# public_key = private_key.publickey()

# # Save the keys
# with open("private.pem", "wb") as f:
#     f.write(private_key.export_key())

# with open("public.pem", "wb") as f:
#     f.write(public_key.export_key())


In [6]:
# import os
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
# from cryptography.hazmat.backends import default_backend
# from Crypto.Cipher import PKCS1_OAEP
# from Crypto.PublicKey import RSA

# # AES encryption for the file
# def aes_encrypt_file(file_path, aes_key):
#     iv = os.urandom(16)  # AES initialization vector
#     cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv), backend=default_backend())
#     encryptor = cipher.encryptor()

#     with open(file_path, 'rb') as f:
#         file_data = f.read()

#     encrypted_data = encryptor.update(file_data) + encryptor.finalize()
#     return iv + encrypted_data

# # RSA encryption for AES key
# def rsa_encrypt_aes_key(aes_key):
#     with open("public.pem", "rb") as f:
#         public_key = RSA.import_key(f.read())
#         rsa_cipher = PKCS1_OAEP.new(public_key)
#         encrypted_aes_key = rsa_cipher.encrypt(aes_key)
#     return encrypted_aes_key

# # Encrypt a file
# file_to_encrypt = "text.txt"
# aes_key = os.urandom(32)  # AES key (256-bit)

# encrypted_file = aes_encrypt_file(file_to_encrypt, aes_key)
# encrypted_aes_key = rsa_encrypt_aes_key(aes_key)

# # Save the encrypted file and AES key
# with open("encrypted_file.bin", "wb") as f:
#     f.write(encrypted_file)

# with open("encrypted_key.bin", "wb") as f:
#     f.write(encrypted_aes_key)

# print("File and AES key encrypted successfully.")


In [7]:
# from Crypto.Cipher import PKCS1_OAEP
# from Crypto.PublicKey import RSA
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# # RSA decryption to get the AES key
# def rsa_decrypt_aes_key(encrypted_aes_key):
#     with open("private.pem", "rb") as f:
#         private_key = RSA.import_key(f.read())
#         rsa_cipher = PKCS1_OAEP.new(private_key)
#         aes_key = rsa_cipher.decrypt(encrypted_aes_key)
#     return aes_key

# # AES decryption for the file
# def aes_decrypt_file(encrypted_data, aes_key):
#     iv = encrypted_data[:16]
#     encrypted_file_data = encrypted_data[16:]

#     cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv), backend=default_backend())
#     decryptor = cipher.decryptor()

#     return decryptor.update(encrypted_file_data) + decryptor.finalize()

# # Decrypt the AES key
# with open("encrypted_key.bin", "rb") as f:
#     encrypted_aes_key = f.read()

# aes_key = rsa_decrypt_aes_key(encrypted_aes_key)

# # Decrypt the file
# with open("encrypted_file.bin", "rb") as f:
#     encrypted_file_data = f.read()

# decrypted_file_data = aes_decrypt_file(encrypted_file_data, aes_key)

# # Save the decrypted file
# with open("decrypted_file.txt", "wb") as f:
#     f.write(decrypted_file_data)

# print("File decrypted successfully.")



In [8]:
import os
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256

# Step 1: Generate RSA keys
passphrase = "hello123"

# Generate RSA keys
private_key = RSA.generate(2048)
public_key = private_key.publickey()

# Ensure the passphrase is of the same length as the key
passphrase_bytes = passphrase.encode()
passphrase_length = len(passphrase_bytes)

# Modify the private key with the passphrase
private_key_bytes = private_key.export_key(format='DER')
modified_key_bytes = bytearray(private_key_bytes)

# XOR each byte of the key with the corresponding byte of the passphrase
for i in range(len(modified_key_bytes)):
    modified_key_bytes[i] ^= passphrase_bytes[i % passphrase_length]

# Save the modified RSA key
with open("modified_private.pem", "wb") as f:
    f.write(bytes(modified_key_bytes))

with open("public.pem", "wb") as f:
    f.write(public_key.export_key())

print("Modified RSA key and public key saved successfully.")


Modified RSA key and public key saved successfully.


In [9]:
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes

# Step 2: Encrypt a File Using AES and RSA
def aes_encrypt_file(file_path, aes_key):
    iv = get_random_bytes(16)  # AES initialization vector
    cipher = AES.new(aes_key, AES.MODE_CFB, iv)

    with open(file_path, 'rb') as f:
        file_data = f.read()

    encrypted_data = iv + cipher.encrypt(file_data)
    return encrypted_data

# RSA encryption for AES key
def rsa_encrypt_aes_key(aes_key):
    with open("public.pem", "rb") as f:
        public_key = RSA.import_key(f.read())
        rsa_cipher = PKCS1_OAEP.new(public_key)
        encrypted_aes_key = rsa_cipher.encrypt(aes_key)
    return encrypted_aes_key

# Encrypt the file
file_to_encrypt = "text.txt"  # Replace with your actual file path
aes_key = get_random_bytes(32)  # AES key (256-bit)

encrypted_file = aes_encrypt_file(file_to_encrypt, aes_key)
encrypted_aes_key = rsa_encrypt_aes_key(aes_key)

# Save the encrypted file and AES key
with open("encrypted_file.bin", "wb") as f:
    f.write(encrypted_file)

with open("encrypted_key.bin", "wb") as f:
    f.write(encrypted_aes_key)

print("File and AES key encrypted successfully.")


File and AES key encrypted successfully.


In [11]:
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256

# Step 3: Decrypt the File Using RSA and AES
def rsa_decrypt_aes_key(encrypted_aes_key, passphrase):
    with open("modified_private.pem", "rb") as f:
        modified_key_bytes = f.read()

    # Ensure the passphrase is of the same length as the key
    passphrase_bytes = passphrase.encode()
    passphrase_length = len(passphrase_bytes)

    # XOR each byte of the stored key with the corresponding byte of the passphrase
    private_key_bytes = bytearray(modified_key_bytes)
    for i in range(len(private_key_bytes)):
        private_key_bytes[i] ^= passphrase_bytes[i % passphrase_length]

    # Import the modified private key
    private_key = RSA.import_key(bytes(private_key_bytes))
    rsa_cipher = PKCS1_OAEP.new(private_key)
    aes_key = rsa_cipher.decrypt(encrypted_aes_key)
    return aes_key

def aes_decrypt_file(encrypted_data, aes_key):
    iv = encrypted_data[:16]
    cipher = AES.new(aes_key, AES.MODE_CFB, iv)

    decrypted_data = cipher.decrypt(encrypted_data[16:])
    return decrypted_data

# Decrypt the AES key
with open("encrypted_key.bin", "rb") as f:
    encrypted_aes_key = f.read()

user_passphrase = input("Enter your secret passphrase: ")
aes_key = rsa_decrypt_aes_key(encrypted_aes_key, user_passphrase)

# Decrypt the file
with open("encrypted_file.bin", "rb") as f:
    encrypted_file_data = f.read()

decrypted_file_data = aes_decrypt_file(encrypted_file_data, aes_key)

# Save the decrypted file
with open("decrypted_file.txt", "wb") as f:
    f.write(decrypted_file_data)

print("File decrypted successfully.")


ValueError: RSA key format is not supported